## Bi Encoder Preprocessing
- Grab sentence and article embeddings using mpnet 

In [1]:
import pandas as pd
import spacy 
from tqdm.auto import tqdm 
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from datasets import Dataset

In [2]:
deviceNum = 4
device = torch.device("cuda:" + str(deviceNum) if torch.cuda.is_available() else "cpu")


In [3]:
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/translatedCleaned.tsv", sep="\t")
df = df.head(20)

In [59]:
df

,Unnamed: 0,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,...,ogText1,ogTitle1,ogText2,ogTitle2,text1,title1,text2,title2,sentences1,sentences2
0,0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.000000,4.000000,...,"MARTINSBURG, W.Va. — A suspected drunken drive...",Virginia man arrested in fatal DUI crash in We...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haiti’s leader marks independence day amid sec...,"MARTINSBURG, W.Va. — A suspected drunken drive...",Virginia man arrested in fatal DUI crash in We...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haiti’s leader marks independence day amid sec...,"[MARTINSBURG, W.Va. —, A suspected drunken dri...","[PORT-AU-PRINCE, Haiti — Haitian President Jov..."
1,1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.000000,4.000000,...,Share This On: Pin 11 Shares (NEWS ROOM GUYA...,Guyana: Three injured after car crashes into u...,BERLIN - A fire at a zoo in western Germany in...,Fire kills more than 30 animals at zoo in west...,Share This On: Pin 11 Shares (NEWS ROOM GUYA...,Guyana: Three injured after car crashes into u...,BERLIN - A fire at a zoo in western Germany in...,Fire kills more than 30 animals at zoo in west...,[Share This On: Pin 11 Shares (NEWS ROOM GUY...,[BERLIN - A fire at a zoo in western Germany i...
2,2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.000000,2.000000,...,(Breitbart) – President Donald Trump welcomed ...,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"PALM BEACH, United States — US President Donal...","Trump says he does not expect war with Iran, ‘...",(Breitbart) – President Donald Trump welcomed ...,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"PALM BEACH, United States — US President Donal...","Trump says he does not expect war with Iran, ‘...",[(Breitbart) – President Donald Trump welcomed...,"[PALM BEACH, United States — US President Dona..."
3,3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber...,https://gadgets.ndtv.com/internet/news/indian-...,https://web.archive.org/web/gadgets.ndtv.com/a...,https://web.archive.org/web/gadgets.ndtv.com/i...,1.000000,2.333333,...,Uber has sold its online food-ordering busines...,Zomato Buys Uber's Food Delivery Business in I...,Rapid digitisation and growth in both online b...,Indian Online Food Delivery Market to Hit $8 B...,Uber has sold its online food-ordering busines...,Zomato Buys Uber's Food Delivery Business in I...,Rapid digitisation and growth in both online b...,Indian Online Food Delivery Market to Hit $8 B...,[Uber has sold its online food-ordering busine...,[Rapid digitisation and growth in both online ...
4,4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-mo...,https://www.channelnewsasia.com/news/asia/indi...,https://web.archive.org/web/news.yahoo.com/ind...,https://web.archive.org/web/www.channelnewsasi...,1.000000,1.250000,...,BENGALURU (Reuters) - India has approved its t...,"India approves third moon mission, months afte...",BANGALORE: India plans to make a fresh attempt...,India targets new moon mission in 2020,BENGALURU (Reuters) - India has approved its t...,"India approves third moon mission, months afte...",BANGALORE: India plans to make a fresh attempt...,India targets new moon mission in 2020,[BENGALURU (Reuters) - India has approved its ...,[BANGALORE: India plans to make a fresh attemp...
5,5,en,en,1484189120_1484113136,https://jewishjournal.com

In [4]:
df.columns

Index(['Unnamed: 0', 'url1_lang', 'url2_lang', 'pair_id', 'link1', 'link2',
       'ia_link1', 'ia_link2', 'Geography', 'Entities', 'Time', 'Narrative',
       'Overall', 'Style', 'Tone', 'id1', 'id2', 'ogText1', 'ogTitle1',
       'ogText2', 'ogTitle2', 'text1', 'title1', 'text2', 'title2'],
      dtype='object')

In [5]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')


In [6]:
#nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"])
#nlp.enable_pipe("senter")
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

def extractSents(inList): 
    separator = nlp.pipe(inList, n_process=8)
    textList = []
    for text in tqdm(separator): 
        sentList = [str(sent) for sent in text.sents if len(sent) > 3]
        textList.append(sentList)
    return textList

df["sentences1"] = extractSents(df["text1"])
df["sentences2"] = extractSents(df["text2"])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [7]:
class CustomSentListDataset(Dataset): 
    def __init__(self, inDf): 
        self.inDf = inDf 
        
    def __len__(self): 
        return len(self.inDf)
    
    def __getitem__(self, idx):
        return list(self.inDf.iloc[idx])

In [8]:
df1 = df[["title1", "sentences1", "url1_lang"]]
df1 = df1.explode("sentences1").reset_index().rename(columns={"index":"article_id"})

df2 = df[["title2", "sentences2", "url2_lang"]]
df2 = df2.explode("sentences2").reset_index().rename(columns={"index":"article_id"})

In [9]:
df1Tokenized = tokenizer.batch_encode_plus(list(df1["sentences1"]), max_length=300, padding="max_length", truncation=True, return_tensors="pt")
df2Tokenized = tokenizer.batch_encode_plus(list(df2["sentences2"]), max_length=300, padding="max_length", truncation=True, return_tensors="pt")

In [10]:
df1["input_ids"] = list(df1Tokenized["input_ids"])
df1["attention_mask"] = list(df1Tokenized["attention_mask"])

df2["input_ids"] = list(df2Tokenized["input_ids"])
df2["attention_mask"] = list(df2Tokenized["attention_mask"])


In [25]:
dataset1 = CustomSentListDataset(df1[["input_ids", "attention_mask"]])
loader1 = torch.utils.data.DataLoader(dataset1, batch_size=5, shuffle=False)

dataset2 = CustomSentListDataset(df2[["input_ids", "attention_mask"]])
loader2 = torch.utils.data.DataLoader(dataset2, batch_size=5, shuffle=False)


In [26]:
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2').to(device)

In [32]:
def mean_pooling(token_embeddings, attention_mask): 
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    

In [33]:
encodings1 = []
for batch in tqdm(loader1): 
    torch.cuda.empty_cache()
    inputs_ids = batch[0].to(device)
    attention = batch[1].to(device)
    encoded = model(inputs_ids, attention)[0]
    mean_pooled = mean_pooling(encoded, attention)
    encodings1 += mean_pooled.detach().to("cpu").tolist()
    del encoded
    del inputs_ids
    del attention
    

encodings2 = []
for batch in tqdm(loader2): 
    torch.cuda.empty_cache()
    inputs_ids = batch[0].to(device)
    attention = batch[1].to(device)
    encoded = model(inputs_ids, attention)[0]
    mean_pooled = mean_pooling(encoded, attention)
    encodings2 += mean_pooled.detach().to("cpu").tolist()
    del encoded
    del inputs_ids
    del attention


  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

In [34]:
df1["encoded"] = encodings1
df2["encoded"] = encodings2

In [61]:
#now we want the cosine similarity between each sentence in each document 
df1 = df1.drop(columns=["input_ids", "attention_mask", "sentences1"])
df1 = df1.groupby(by="article_id").agg({"title1":"first", "encoded":list, "url1_lang":"first"})

df2 = df2.drop(columns=["input_ids", "attention_mask", "sentences2"])
df2 = df2.groupby(by="article_id").agg({"title2":"first", "encoded":list, "url2_lang":"first"})

merged = pd.merge(df1, df2, how="inner", on="article_id")

In [68]:
merged["encoded_y"]

article_id
0     [[0.0061991456896066666, 0.18428094685077667, ...
1     [[-0.0048051755875349045, 0.13477689027786255,...
2     [[0.012652377597987652, 0.16973021626472473, 0...
3     [[0.02911442331969738, 0.20269231498241425, -0...
4     [[0.19905565679073334, 0.08663235604763031, 0....
5     [[-0.02054200880229473, 0.21008895337581635, 0...
6     [[-0.14503853023052216, -0.047894056886434555,...
7     [[0.014706699177622795, 0.062391333281993866, ...
8     [[0.03750452399253845, 0.18643204867839813, 0....
9     [[-0.023145105689764023, 0.06932254880666733, ...
10    [[0.05261794105172157, 0.11548543721437454, -0...
11    [[-0.07172594964504242, 0.0352322943508625, 0....
12    [[0.03410551697015762, 0.12198571860790253, -0...
13    [[-0.06259718537330627, -0.03564434498548508, ...
14    [[-0.050275564193725586, 0.24223309755325317, ...
15    [[-0.05258717015385628, -0.12546539306640625, ...
16    [[0.0061991456896066666, 0.18428094685077667, ...
17    [[0.00185864744707942, -0.00685

In [72]:
#TODO: potentially normalize the vectors? 

from scipy import spatial 
def getTopSim(l1, l2): 
    simList = []
    for v1 in l1: 
        mostSim = 0
        for v2 in l2: 
            sim = spatial.distance.cosine(v1, v2)
            if sim > mostSim: 
                mostSim = sim
        simList.append(mostSim)
    return simList

l1 = merged["encoded_y"][0]
l2 = merged["encoded_x"][0]
getTopSim(l1, l2)

[1.0471863035284963,
 1.0130859889945052,
 1.051935360822316,
 1.0205879245335328,
 1.0756650975768967,
 1.0375394339890818,
 1.0072212437438697,
 1.0544481716564835,
 1.0528239911110073,
 1.033451155011668,
 1.0137188718252823,
 1.0774919930813862,
 1.0772467686747687]

In [76]:
simVecs = merged.apply(lambda x: getTopSim(x["encoded_x"], x["encoded_y"]), axis=1)

In [77]:
merged["simVecs"] = simVecs

In [78]:
merged

,title1,encoded_x,url1_lang,title2,encoded_y,url2_lang,simVecs
article_id,,,,,,,
0,Virginia man arrested in fatal DUI crash in We...,"[[-0.06316892802715302, 0.12605959177017212, 0...",en,Haiti’s leader marks independence day amid sec...,"[[0.0061991456896066666, 0.18428094685077667, ...",en,"[1.007269701756004, 1.0774919930813862, 1.0205..."
1,Guyana: Three injured after car crashes into u...,"[[0.12181060761213303, 0.0504269152879715, 0.0...",en,Fire kills more than 30 animals at zoo in west...,"[[-0.0048051755875349045, 0.13477689027786255,...",en,"[1.0030139784954455, 1.0527066214838288, 1.035..."
2,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"[[-0.04077819362282753, 0.10680670291185379, 0...",en,"Trump says he does not expect war with Iran, ‘...","[[0.012652377597987652, 0.16973021626472473, 0...",en,"[0.9213849724988418, 0.9479399429024488, 1.027..."
3,Zomato Buys Uber's Food Delivery Business in I...,"[[0.03576498106122017, 0.05767017602920532, -0...",en,Indian Online Food Delivery Market to Hit $8 B...,"[[0.02911442331969738, 0.20269231498241425, -0...",en,"[0.9884018179625843, 1.0196353379697434, 1.019..."
4,"India approves third moon mission, months afte...","[[0.2549508213996887, -0.06316936016082764, -0...",en,India targets new moon mission in 2020,"[[0.19905565679073334, 0.08663235604763031, 0....",en,"[0.9034620488034544, 0.8699847252323691, 0.937..."
5,Wiesenthal Center Calls for FBI Task Force to ...,"[[0.049509208649396896, 0.1936918944120407, 0....",en,Jewish Groups React to Monsey Stabbings,"[[-0.02054200880229473, 0.21008895337581635, 0...",en,"[0.8575301394631156, 0.9377098545480901, 0.829..."
6,Big cut on import duty on crude and refined pa...,"[[-0.04981398954987526, -0.048491720110177994,...",en,Delhi weather update: Delhiites shiver at 2.4 ...,"[[-0.14503853023052216, -0.047894056886434555,...",en,"[1.0520425137227154, 1.0275731040104685, 0.953..."
7,The biggest business con of 2019: fleecing wor...,"[[-0.06818566471338272, 0.2567267119884491, 0....",en,The left’s constant victimhood olympics is a g...,"[[0.014706699177622795, 0.062391333281993866, ...",en,"[1.0676002452141853, 1.1015170800481293, 1.070..."
8,The Compatriots: The Brutal and Chaotic Histor...,"[[-0.022116590291261673, 0.12838920950889587, ...",en,The Red Web: The Kremlin's Wars on the Interne...,"[[0.03750452399253845, 0.18643204867839813, 0....",en,"[0.9332892746163253, 0.806967111957389, 0.6688..."


In [ ]:
#torch.flatten(encodings1[0])
#try to get all embeddings in a list one after another, then we add that to dataframe as a column, and do 
#a groupby on the article id 
outlist = []
for item in encodings1: 
    outlist.append(np.array(item.cpu()))
outlist = np.array(outlist)
outlist

In [ ]:
#NOTE: reference this later 
testEncoding = model(moddedDf.loc[1, "text1_input_ids"], max_length = 200, attention_mask=moddedDf.loc[1, "text1_attention_mask"])[0]

In [ ]:
for i, row in tqdm(moddedDf.iterrows()): 
    sentEncodings1 = model(row["text1_input_ids"], max_length = 200, attention_mask=row["text1_attention_mask"])[0]

In [ ]:
def embedSents(): 
    encodedList = [model() for item in inList]
    return encodedList

In [ ]:
moddedDf.head(3)
leanDf = moddedDf

In [ ]:
#TODO: pool all of the encodings to get one per sentence. Put this into an extra column in df
#TODO: do the same for the title and the first part of each article 

In [ ]:
testEncoding.shape

In [ ]:
def encode(inList): 
    return []

In [ ]:
extracted1[0]